In [ ]:
from torch import nn
import torch
import logging
from collections import OrderedDict
import models.config as cfg

In [ ]:
def make_layers(block):
  """
  Function for creating layers used for preparing the input before feeding to ConvLSTM
  """
  layers = []
  for layer_name, v in block.items():
    if 'conv' in layer_name:
      conv2d = nn.Conv2d(in_channels=v[0], out_channels=v[1],
                               kernel_size=v[2], stride=v[3],
                               padding=v[4])
      layers.append((layer_name, conv2d))
            ## Instead of using ReLU; here we used LeakyReLU
      layers.append(('leaky_' + layer_name, nn.LeakyReLU(negative_slope=0.2, inplace=True)))
    elif 'deconv' in layer_name:
      transposeConv2d = nn.ConvTranspose2d(in_channels=v[0], out_channels=v[1],
                                                 kernel_size=v[2], stride=v[3],
                                                 padding=v[4])
      layers.append((layer_name, transposeConv2d))
            ## Instead of using ReLU; here we used LeakyReLU
      layers.append(('leaky_' + layer_name, nn.LeakyReLU(negative_slope=0.2, inplace=True)))
    else:
      raise NotImplementedError
  return nn.Sequential(OrderedDict(layers))


In [ ]:
class Encoder(nn.Module):
  """
  The class represent the Encoder part used for encode the input into features map used for Forecaster.
  """
  def __init__(self, subnets, rnns):
    """
    Initiate the Encoder.
    The parameter used here is Subnet and RNN.

    Encoder consist of several stages which each of them consist of  Subnet and RNN Layers. 
    Subnet used for preparing the input target into desired input matrix which further used in RNN.
    RNN processing the input for producinng the hidden state and output
    -----
    Parameter:
    - Subnet: list of layer used for preparing the input for the RNN
    - RNN: list of layer for predicting task
    """ 
    super().__init__()
    # Check wheter the number of item in subnet equals to number of RNN layer
    assert len(subnets)==len(rnns)
    self.blocks = len(subnets)

    # Initiate the stage and RNN layer
    for index, (params, rnn) in enumerate(zip(subnets, rnns), 1):
      # Stage represent the network used for preparing the input 
      setattr(self, 'stage'+str(index), make_layers(params))
      # RNN represent the network used for processing the input feature
      setattr(self, 'rnn'+str(index), rnn)

  def forward_by_stage(self, input, subnet, rnn):
    """
    Function for proccessing the "STAGE" 
    """
    seq_number, batch_size, input_channel, height, width = input.size()
    input = torch.reshape(input, (-1, input_channel, height, width))
    # Preparing the input
    input = subnet(input)
    input = torch.reshape(input, (seq_number, batch_size, input.size(1), 
                                      input.size(2), input.size(3)))
    # Processing the prepared input
    outputs_stage, state_stage = rnn(input, None)
    return outputs_stage, state_stage

  # input: 5D S*B*I*H*W
  def forward(self, input):
    """
    Feed forwarding the Encoding layer
    """
    hidden_states = []
    logging.debug(input.size())
    for i in range(1, self.blocks+1):
      input, state_stage = self.forward_by_stage(input, getattr(self, 'stage'+str(i)), 
                                                       getattr(self, 'rnn'+str(i)))
      hidden_states.append(state_stage)
    return tuple(hidden_states)

In [ ]:
class Forecaster(nn.Module):
  """
  The class represent the Forecaster part used for forecasting the features map from Encoder
  """
  def __init__(self, subnets, rnns):
    """
    Initiate the Forecaster.
    The parameter used here is Subnet and RNN.

    As similar with Encoder, Forecaster consist of several stages which each of them consist of  Subnet and RNN Layers. 
    Contrary to the Encoder, here the flow is reversed from the RNN to Subnet. 
    RNN processing the hidden state for producinng the hidden state and output
    Subnet used for translating the result into features which similar to the input.
    -----
    Parameter:
    - Subnet: list of layer used for tranlsation task
    - RNN: list of layer for predicting task
    """ 
    super().__init__()
    # Check wheter the number of item in subnet equals to number of RNN layer
    assert len(subnets) == len(rnns)
    self.blocks = len(subnets)


    # Initiate the stage and RNN layer
    for index, (params, rnn) in enumerate(zip(subnets, rnns)):
      # RNN represent the network used for forecasting the hidden state
      setattr(self, 'rnn' + str(self.blocks-index), rnn)
      # Stage represent the network used for translation
      setattr(self, 'stage' + str(self.blocks-index), make_layers(params))

  def forward_by_stage(self, input, state, subnet, rnn,seq_len=cfg.sequence_len['OUT']):
    """
    Function for proccessing the "STAGE" 
    """
    # by default the number of predicting is set to 10
    # Forecasting task
    input, state_stage = rnn(input, state, seq_len)
    seq_number, batch_size, input_channel, height, width = input.size()
    input = torch.reshape(input, (-1, input_channel, height, width))
    # translation task
    input = subnet(input)
    input = torch.reshape(input, (seq_number, batch_size, input.size(1), input.size(2), input.size(3)))

    return input
  
  # input: 5D S*B*I*H*W
  def forward(self, hidden_states):
    """
    Feed forwarding the Encoding layer
    """
    input = self.forward_by_stage(None, hidden_states[-1], getattr(self, 'stage'+str(self.blocks)),
                                      getattr(self, 'rnn'+str(self.blocks)))
    for i in list(range(1, self.blocks))[::-1]:
      input = self.forward_by_stage(input, hidden_states[i-1], getattr(self, 'stage' + str(i)),
                                                       getattr(self, 'rnn' + str(i)))
    return input


In [ ]:
class EF(nn.Module):
  """
  The class represent the encapsulation of Encoder part and Decoder part
  """
  def __init__(self, encoder, forecaster):
    super().__init__()
    self.encoder = encoder
    self.forecaster = forecaster

  def forward(self, input):
    state = self.encoder(input)
    output = self.forecaster(state)
    return output